In [2]:
import classla
import pandas as pd
import numpy as np
#classla.download('hr')
#classla.download('hr', type='nonstandard')

In [75]:
lyrics_df = pd.read_csv('data.csv')
lyrics_df = lyrics_df[lyrics_df.Artist=="Severina"]
lyrics_df = lyrics_df.dropna().Lyrics


In [76]:
lyrics = [l.splitlines() for l in lyrics_df[0:50]]

In [78]:
all_lyrics = [" ".join(l) for l in lyrics]

In [79]:
#classla.download('hr', type='nonstandard')
nlp = classla.Pipeline('hr', type='nonstandard') # run classla.download('hr', type='nonstandard') beforehand if necessary
doc = nlp(".".join(all_lyrics))



2022-02-16 18:03:29 INFO: Loading these models for language: hr (Croatian):
| Processor | Package     |
---------------------------
| tokenize  | nonstandard |
| pos       | nonstandard |
| lemma     | nonstandard |
| depparse  | standard    |
| ner       | nonstandard |

2022-02-16 18:03:29 INFO: Use device: cpu
2022-02-16 18:03:29 INFO: Loading: tokenize
2022-02-16 18:03:29 INFO: Loading: pos
2022-02-16 18:03:30 INFO: Loading: lemma
2022-02-16 18:03:36 INFO: Loading: depparse
2022-02-16 18:03:37 INFO: Loading: ner
2022-02-16 18:03:37 INFO: Done loading processors!


In [89]:
doc_dict = doc.to_dict()

In [90]:
df = pd.DataFrame([doc_dict[i][0] for i, _ in enumerate(doc_dict)]).transpose()

In [91]:
sentences = [doc_dict[i][0] for i, _ in enumerate(doc_dict)]
df = pd.DataFrame(sentences[0])
for sentence in sentences[1:]:
    df = pd.concat([df, pd.DataFrame(sentence)])

In [92]:
df.reset_index(drop=False, inplace=True)

In [35]:
pos_tags = "ADJ – ADP – ADV – AUX – CCONJ – DET – INTJ – NOUN – NUM – PART – PRON – PROPN – PUNCT – SCONJ – SYM – VERB – X".split(" – ")
relations = "acl – advcl – advmod – advmod:emph – amod – appos – aux – case – cc – ccomp – compound – conj – cop – csubj – dep – det – det:numgov – discourse – dislocated – expl – fixed – flat – flat:foreign – iobj – list – mark – nmod – nsubj – nummod – obj – obl – orphan – parataxis – punct – root – vocative – xcomp".split(" – ")
word_features = "Animacy – Case – Definite – Degree – Foreign – Gender – Gender[psor] – Mood – Number – Number[psor] – NumType – Person – Polarity – Poss – PronType – Reflex – Tense – VerbForm – Voice".split(" – ")

In [93]:
features_df = pd.DataFrame()
rows = df.feats.str.split("|").to_list()

for i, row in enumerate(rows):
    if not type(row) == float:
        for f in row:
            features_df.loc[i, f[0:f.index("=")]] = f[f.index("=")+1:]


In [94]:
df = df.join(features_df)

In [95]:
df.Case.unique()

array([nan, 'Nom', 'Ins', 'Loc', 'Gen', 'Acc', 'Dat', 'Voc'], dtype=object)

In [97]:
df[(df.upos=="VERB")]["lemma"].value_counts()[0:10]

dati       47
ljubiti    37
voljeti    37
znati      36
imati      27
otići      19
krenuti    16
gledati    15
moći       14
spavati    13
Name: lemma, dtype: int64